## import

In [2]:

import pandas as pd
import numpy as np
import pathlib
from pathlib import Path
from itertools import cycle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import circlify
from matplotlib.ticker import PercentFormatter
from matplotlib import ticker
import matplotlib as mpl
import matplotlib.cm as cm
import plotly.express as px

In [ ]:

Directory = '/Users/yaale244/Desktop/Alex19022023/CSBCIPredictionAlgorithm/tcDCS post stim/2s30-31npwcut/np' #change directory as needed

pathlist = Path(Directory).rglob('*.csv')
for path in pathlist:
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    l = len(filename)
    filename = filename[0:l-4]
    df = pd.read_table(path, header = None, delimiter = ',') # reads table including first row
    percentage = df[1].tolist()
    print(percentage)

    #get a color list, palette name can be changed
    hexlist = []
    for x in percentage:
        norm = mpl.colors.Normalize(vmin=df[1].min(), vmax=df[1].max())
        cmap = cm.coolwarm_r
        # cmap = sns.diverging_palette(14,254, l=50, s=75, as_cmap=True)
        m = cm.ScalarMappable(norm=norm, cmap=cmap)
        rbga = (m.to_rgba(x))
        hex = (mpl.colors.to_hex((rbga), keep_alpha=True))
        hexlist.append(hex)
    print(hexlist)
    
    # compute circle positions:
    
    circles = circlify.circlify(percentage, show_enclosure=False, target_enclosure=circlify.Circle(x=0, y=0))
    circles.reverse()

    #create a laebls list for each bubble 
    label = [i+'<br>'+str(k)+'%' for i,k in zip(df[0], df[1])]
    fig, ax = plt.subplots(figsize=(14,14), facecolor='white')
    ax.axis('off')
    lim = max(max(abs(circle.x)+circle.r, abs(circle.y)+circle.r,) for circle in circles)
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)

    # print circles
    
    for circle, note, color in zip(circles, label, hexlist):
        x, y, r = circle
        ax.add_patch(plt.Circle((x, y), circle.r, alpha=1, color=color))
        plt.annotate(note.replace('<br>','\n'), (x,y), size=16, va='center', ha='center')
    plt.xticks([])
    plt.yticks([])
    if df[1].max()!=43:
        norm = plt.Normalize(df[1].min(), df[1].max())
        tick_locator = ticker.MaxNLocator(nbins = 5)
    else:
        norm = plt.Normalize(df[1].min(), df[1].max()+5)
        tick_locator = ticker.MaxNLocator(nbins = 10)
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    
    cax = fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.03, ax.get_position().height / 2])
    cb = ax.figure.colorbar(sm, cax=cax)
    cb.locator = tick_locator
    cb.update_ticks()
    
    plt.savefig(filename+'.tiff', dpi=600, bbox_inches='tight')
    plt.show()
    

In [26]:
import plotly.express as px
import pandas as pd
import plotly.io as pio

Directory = '/Users/yaale244/Desktop/Alex19022023/CSBCIPredictionAlgorithm/tDCS baseline/2s30-31npwcut/np/base' #change directory as needed

pathlist = Path(Directory).rglob('*.csv')
for path in pathlist:
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    l = len(filename)
    filename = filename[0:l-4]
    df = pd.read_table(path, header=None, delimiter=',') # reads table including first row
    df.columns = ['region', 'percentage']
    print(filename)
    fig = px.scatter(df, x='region', y='percentage', size='percentage', color='percentage', size_max=50, text='region')
    fig.update_traces(textposition='top center')
    fig.update_layout(
        title='',
        xaxis_title="Regions",
        yaxis_title="Neuron Proportion",
        yaxis=dict(range=[0,15]),
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="black"
        ),
        width=1050, # increase plot width
        height=500, # increase plot height
        xaxis_tickangle=-90, # rotate x-axis labels for better readability
        xaxis_tickfont=dict(size=14), # adjust x-axis label font size
        margin=dict(l=80, r=50, t=80, b=50) # adjust plot margins
    )

# Export the figure as a PNG image
    fig.write_html(f"{filename}.html", include_plotlyjs='cdn')
    # Save the figure in PNG format with 600 DPI
    fig.write_image(f"{filename}.png", engine='kaleido', scale=6)
  
    fig.show()


1placebobase


2anodalbase


3cathodalbase


In [85]:
import plotly.express as px
import pandas as pd
import plotly.io as pio
from pathlib import Path

Directory = '/Users/yaale244/Desktop/Alex19022023/CSBCIPredictionAlgorithm/tDCS baseline/2s30-31npwcut/np/subtractedbasestim' #change directory as needed

pathlist = Path(Directory).rglob('*.csv')
dfs = [] # initialize list to store data frames
for path in sorted(pathlist):
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    l = len(filename)
    filename = filename[0:l-4]
    df = pd.read_table(path, header=None, delimiter=',') # reads table including first row
    df.columns = ['region', 'percentage']
    df['time'] = filename # add time column to data frame
    dfs.append(df) # add data frame to list
df_concat = pd.concat(dfs) # concatenate data frames into one

fig = px.scatter(df_concat, x='region', y='percentage', color='percentage', size=df_concat['percentage'].abs(), size_max=50, text=np.where(df_concat['percentage'] != 0, df_concat['region'], ''), color_continuous_scale='plasma', animation_frame='time')
fig.update_traces(text=df_concat['percentage'].apply(lambda x: f'{x:.0f}' if abs(x) >= 1 else ''), selector=dict(mode='markers'))
fig.update_traces(
    textposition='top center')
fig.update_layout(
    yaxis=dict(range=[-7,7]),
    title='',
    xaxis_title="Regions",
    yaxis_title="Neuron Proportion",
    xaxis_categoryorder='array',
    xaxis_categoryarray=list(df_concat['region'].unique()), # set the order of x-axis labels
    xaxis={'categoryorder': 'array',
           'categoryarray': list(df['region'])}, # set the order of x-axis labels
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="black"
    ),
    width=1050, # increase plot width
    height=500, # increase plot height
    xaxis_tickangle=-90, # rotate x-axis labels for better readability
    xaxis_tickfont=dict(size=14), # adjust x-axis label font size

    margin=dict(l=80, r=50, t=80, b=50), # adjust plot margins
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        buttons=[dict(
            label='Play',
            method='animate',
            args=[None, {'frame': {'duration': 1000}, 'fromcurrent': True}]
        ), dict(
            label='Pause',
            method='animate',
            args=[[None], {'frame': {'duration': 0}, 'mode': 'immediate', 'transition': {'duration': 0}}]
        )]
    )]
)

# Export the figure as a HTML file
fig.write_html("animationdelta.html", include_plotlyjs='cdn')
# Save each animation frame as a PNG image
for i in range(len(df_concat['time'].unique())):
    fig.update_layout(title_text=f"Time = {df_concat['time'].unique()[i]}") # set title for current frame
    filename = df_concat['time'].unique()[i] + ".png"
    fig.write_image(filename, engine='kaleido', scale=6)
# Show the figure
fig.show()


In [93]:
import plotly.express as px
import pandas as pd
import plotly.io as pio
from pathlib import Path

Directory = '/Users/yaale244/Desktop/Alex19022023/CSBCIPredictionAlgorithm/tDCS baseline/2s30-31npwcut/np/stimcomparisons' #change directory as needed

pathlist = Path(Directory).rglob('*.csv')
dfs = [] # initialize list to store data frames
for path in sorted(pathlist):
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    l = len(filename)
    filename = filename[0:l-4]
    df = pd.read_table(path, header=None, delimiter=',') # reads table including first row
    df.columns = ['region', 'percentage']
    df['time'] = filename # add time column to data frame
    dfs.append(df) # add data frame to list
df_concat = pd.concat(dfs) # concatenate data frames into one

fig = px.scatter(df_concat, x='region', y='percentage', color='percentage', size=df_concat['percentage'].abs(), size_max=50, text=np.where(df_concat['percentage'] != 0, df_concat['region'], ''), color_continuous_scale='plasma', animation_frame='time')
fig.update_traces(text=df_concat['percentage'].apply(lambda x: f'{x:.0f}' if abs(x) >= 1 else ''), selector=dict(mode='markers'))
fig.update_traces(
    textposition='top center')
fig.update_layout(
    yaxis=dict(range=[-11,11], dtick=1),
    title='',
    xaxis_title="Regions",
    yaxis_title="Neuron Proportion",
    xaxis_categoryorder='array',
    xaxis_categoryarray=list(df_concat['region'].unique()), # set the order of x-axis labels
    xaxis={'categoryorder': 'array',
           'categoryarray': list(df['region'])}, # set the order of x-axis labels
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="black"
    ),
    width=1050, # increase plot width
    height=500, # increase plot height
    xaxis_tickangle=-90, # rotate x-axis labels for better readability
    xaxis_tickfont=dict(size=14), # adjust x-axis label font size

    margin=dict(l=80, r=50, t=80, b=50), # adjust plot margins
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        buttons=[dict(
            label='Play',
            method='animate',
            args=[None, {'frame': {'duration': 1000}, 'fromcurrent': True}]
        ), dict(
            label='Pause',
            method='animate',
            args=[[None], {'frame': {'duration': 0}, 'mode': 'immediate', 'transition': {'duration': 0}}]
        )]
    )]
)

# Export the figure as a HTML file
fig.write_html("animationdelta.html", include_plotlyjs='cdn')
# Save each animation frame as a PNG image
for i in range(len(df_concat['time'].unique())):
    fig.update_layout(title_text=f"Time = {df_concat['time'].unique()[i]}") # set title for current frame
    filename = df_concat['time'].unique()[i] + ".png"
    fig.write_image(filename, engine='kaleido', scale=6)
# Show the figure
fig.show()

